In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget https://dl-challenge.zalo.ai/e2e-question-answering/e2eqa-train+public_test-v1.zip
!wget https://dl-challenge.zalo.ai/e2e-question-answering/wikipedia_20220620_cleaned.zip

--2023-08-28 07:15:50--  https://dl-challenge.zalo.ai/e2e-question-answering/e2eqa-train+public_test-v1.zip
Resolving dl-challenge.zalo.ai (dl-challenge.zalo.ai)... 49.213.78.231
Connecting to dl-challenge.zalo.ai (dl-challenge.zalo.ai)|49.213.78.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3950125 (3.8M) [application/zip]
Saving to: ‘e2eqa-train+public_test-v1.zip’

e2eqa-train+public_ 100%[===================>]   3.77M  2.65MB/s    in 1.4s    

2023-08-28 07:15:52 (2.65 MB/s) - ‘e2eqa-train+public_test-v1.zip’ saved [3950125/3950125]

--2023-08-28 07:15:52--  https://dl-challenge.zalo.ai/e2e-question-answering/wikipedia_20220620_cleaned.zip
Resolving dl-challenge.zalo.ai (dl-challenge.zalo.ai)... 49.213.78.231
Connecting to dl-challenge.zalo.ai (dl-challenge.zalo.ai)|49.213.78.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 417016977 (398M) [application/zip]
Saving to: ‘wikipedia_20220620_cleaned.zip’

wikipedia_20220620_

In [2]:
!wget https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords-dash.txt

--2023-08-29 01:29:21--  https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords-dash.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20475 (20K) [text/plain]
Saving to: ‘vietnamese-stopwords-dash.txt’

vietnamese-stopword 100%[===================>]  20.00K  --.-KB/s    in 0.001s  

2023-08-29 01:29:21 (16.2 MB/s) - ‘vietnamese-stopwords-dash.txt’ saved [20475/20475]



In [ ]:
!unzip -q /content/e2eqa-train+public_test-v1.zip
!unzip -q /content/wikipedia_20220620_cleaned.zip

In [3]:
!pip install pyvi
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.6 MB/s eta 0:00:00


In [4]:
import json
import torch
from tqdm import tqdm
from pyvi import ViTokenizer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

In [5]:
stopword_file = open('vietnamese-stopwords-dash.txt', 'r')
stopword = stopword_file.read()
stopword = stopword.split('\n')

In [6]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("hogger32/xlmRoberta-for-VietnameseQA")
model = AutoModelForQuestionAnswering.from_pretrained("hogger32/xlmRoberta-for-VietnameseQA")

qa_model = pipeline("question-answering", model="hogger32/xlmRoberta-for-VietnameseQA")


In [7]:
!pip install rank_bm25
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=1be0d3aaef10e7973322d3df3d0123a59714fbbe4663a2900efc2b50f376b659
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [12]:
from rank_bm25 import BM25Okapi
# from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder, util

# passages = []
# for p in data["data"]:
#     passages.append(p["text"])
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# We lower case our text and remove stop-words from indexing
def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in stopword:
            tokenized_doc.append(token)
    return tokenized_doc

# tokenized_corpus = []
# for passage in tqdm(passages):
#     tokenized_corpus.append(bm25_tokenizer(passage))

Test trên Dataset Chatbot cty

In [8]:
import pandas as pd
my_data = pd.read_excel('/content/drive/MyDrive/final_chatbot_dataset.xlsx')

In [9]:
my_contexts = my_data['Context'].tolist()
my_questions = my_data['Question'].tolist()
my_answers = my_data['Answer'].tolist()

In [10]:
# my_contexts.pop(648)
# my_questions.pop(648)
# my_answers.pop(648)

In [13]:
my_corpus = []
for context in tqdm(my_contexts):
  my_corpus.append(bm25_tokenizer(context))

  0%|          | 0/1510 [00:00<?, ?it/s]

In [14]:
my_bm25 = BM25Okapi(my_corpus)

In [21]:
def my_search(query):
#     print("Input question:", query)

    ##### BM25 search (lexical search) #####
    bm25_scores = my_bm25.get_scores(bm25_tokenizer(query))
    # print(bm25_scores)
    top_n = np.argpartition(bm25_scores, -5)[-5:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

    # print("Top-3 lexical search (BM25) hits")
    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, my_contexts[hit['corpus_id']]] for hit in bm25_hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        bm25_hits[idx]['cross-score'] = cross_scores[idx]

    bm25_hits = sorted(bm25_hits, key=lambda x: x['cross-score'], reverse=True)

    results = []
    for hit in bm25_hits[0:5]:
#         print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " ")))
        results.append(my_contexts[hit['corpus_id']].replace("\n", " "))
    return results

In [16]:
print(my_search(' Đối với công tác triển khai xây lắp, lắp đặt hệ thống hạ tầng kỹ thuật, thiết bị (bao gồm hiệu chỉnh vật tư, thiết bị) và cài đặt phần mềm thương mại trong điều 4 thông tư số 24/2020/TT-BTTTT cần làm những gì?'))

[14.72719822 14.72719822 14.93479567 ...  5.84793363  5.84793363
  4.89717462]
['"Điều 4. Thực hiện công tác triển khai của thông tư số 24/2020/TT-BTTTT Nhà thầu triển khai thực hiện các công việc sau: 1. Lập bảng tiến độ thực hiện chi tiết trước khi triển khai theo quy định về quản lý tiến độ thực hiện dự án tại Điều 33 Nghị định số 73/2019/NĐ-CP. 2. Lập hệ thống quản lý chất lượng phù hợp với quy mô triển khai (nếu chủ đầu tư yêu cầu), trong đó nêu rõ trách nhiệm của từng bộ phận, cá nhân trong quá trình triển khai. 3. Bố trí nhân lực, thiết bị triển khai theo hợp đồng. 4. Đối với công tác triển khai xây lắp, lắp đặt hệ thống hạ tầng kỹ thuật, thiết bị (bao gồm hiệu chỉnh vật tư, thiết bị) và cài đặt phần mềm thương mại  a) Tổ chức thực hiện kiểm tra chất lượng, số lượng, chủng loại của sản phẩm, thiết bị trước khi bàn giao cho chủ đầu tư; b) Cung cấp cho chủ đầu tư các chứng chỉ, chứng nhận, các thông tin, tài liệu có liên quan tới sản phẩm, thiết bị, phần mềm thương mại theo quy đị

In [17]:
q = my_questions[200]
c = my_contexts[200]
r = qa_model(question = q, context = c)

In [19]:
print(q)

Trong thành phần hao phí nhân công tính đến công của chuyên gia nước ngoài đối với những công việc cần chuyên gia chưa?


In [18]:
print(r)

{'score': 0.5405251383781433, 'start': 967, 'end': 971, 'answer': 'chưa'}


In [ ]:
bot_answer = []
for question in tqdm(my_questions):
  retrieval_result = my_search(question)
  max_score = 0
  for retrieval in retrieval_result:
    result = qa_model(question = question, context = retrieval)
    score = result['score']
    if score > max_score:
      max_score = score
      max_result = result['answer']
  bot_answer.append(max_result)

  0%|          | 0/1510 [00:00<?, ?it/s]

In [ ]:
print(bot_answer)

['Libya', 'BOT.', 'Octet', 'Thụy Điển-Mỹ', 'thế chiến thứ hai', 'Hội Minh Tân', 'Hội Minh Tân', 'México bị thực dân Tây Ban Nha', 'México bị thực dân Tây Ban Nha', 'không nhất thiết phải đầy đủ ba bộ phận trong một quy phạm pháp luật.', 'Minh Trị,', 'Mặt Trăng,', 'Iran', 'Chính phủ Mỹ', 'Cộng hòa Nhân dân Trung Hoa', 'Linh Linh', 'mở rộng thương mại toàn cầu đối với các sản phẩm CNTT', 'Toa án Công lý Liên minh châu Âu', 'Chính phủ Mỹ', 'Einstein', 'Worldwide"BBC Worldwide Limited là một công ty con của BBC,', 'triều đại Mãn Thanh,', 'Trinity', 'Uzbekistan', 'CHDCND Triều Tiên', 'Hai Bà Trưng.', 'Nhật', 'Ngày 20 tháng 7 năm 1962,', 'thế kỉ 22 trở về thế kỉ 20', 'Ai Cập', 'Einstein', 'nhà Minh', 'người Tày', 'Hai Bà Trưng.', 'nước nào và cơ chế', 'Cộng hoà Dân chủ Nhân dân Triều Tiên', 'Mặt Trăng,', 'Viện kiểm sát nhân dân tối cao', 'Hội Minh Tân', 'Thủ trưởng cơ quan CSĐT - BCA', 'Thủ trưởng cơ quan CSĐT - BCA', 'Bộ Tài nguyên và Môi trường;', 'Tổng cục Địa chất và Khoáng sản', '"Ban K